# Université Paul Sabatier

M1IAFA - Recherche d'information

**TP 2**

Enseignants : Lynda Tamine et Jesús Lovón

Notebook proposé par : José G. Moreno


2024

---

💡 Penser à développer des scripts et fonctions auxiliaires qui vont permettre réutiliser les commandes récurrentes dans ce TP et les prochains. Ceci aussi vous permettra de garder de bon pratique du code et faciliter vos débogages.

---


### Attention ❗❗ Pour la note du TP :
🚨 *Questions de code* : Remplir le code manquant dans les parties correspondantes (le code commenté bénéficie des meilleures notes).

🚨 *Questions ouvertes* : Ecrivez votre réponse textuelle sous forme de commentaires dans les cellules correspondantes.

🚨 *Laissez vos sorties* pour les cellules où vous écrivez le code. Les sorties vides (notebook ou cellules non exécutées) correspondent à 0 points.

---

# TP 2. Application de la RI pour la liaison référentielle

Une application des méthodes de RI est la liaison référentielle. Cette tâche consiste à trouver le lien entre une portion de texte et la page Wikipédia qui le représentent le mieux. Dans ce cas d’application :
* Comme requêtes nous allons utiliser chaque mot dans une phrase (ou chaque mot important).
* Le corpus sont les pages Wikipédia. Donc, chaque page Wikipédia est un document à retrouver.
* La liste de résultats sont les candidats pour la liaison référentielle.

Considère par exemple la chaîne de caractères « ***Thomas and Mario are strikers playing in Munich*** ». Il existe trois sous-chaînes (***Thomas***, ***Mario*** et ***Munich***) ou mots importants qui sont susceptibles d'être identifiées dans une base de connaissances telle que Wikipédia. Une recherche de ces mots-clés dans les pages Wikipédia produirait de nombreux concepts candidats pour ces sous-chaînes. Donc un bon système doit retrouver comme premier résultats les liens aux pages Wikipédia : [Thomas Müller](https://en.wikipedia.org/wiki/Thomas_M%C3%BCller), [Mario Gómez](https://en.wikipedia.org/wiki/Mario_G%C3%B3mez) (ou Mario Götze) et FC [Bayern Munich](https://en.wikipedia.org/wiki/FC_Bayern_Munich). Cette tâche est particulièrement complexe et les méthodes de recherche d’information sont particulièrement utiles pour la résoudre.

> Plusieurs méthodes de liaison référentielle (Entity Linking en anglais) sont disponibles sur internet. Les plus connus sont :

>	* Spotlight → https://demo.dbpedia-spotlight.org/
>	* Babelfy → http://babelfy.org/
>	* TagMe → https://tagme.d4science.org/tagme/

Étapes à suivre pendant ce TP :
1. Liaison avec le texte
2. Les poids
3. Utilisation de termes
4. Le titre
5. L'outil


## 0. Installation de l’environnement de travail

Installation et configuration de librairies comment le TP précédent.

In [ ]:
# déclaration de la variable JAVA_HOME
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'
!export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64

In [ ]:
#installation de pyterrier avec pip
!pip install --upgrade git+https://github.com/terrier-org/pyterrier.git#egg=python-terrier

  Cloning https://github.com/terrier-org/pyterrier.git to /tmp/pip-install-y69mhc__/python-terrier_a0c45474fa0b4352823c34dd2c163061
  Running command git clone --filter=blob:none --quiet https://github.com/terrier-org/pyterrier.git /tmp/pip-install-y69mhc__/python-terrier_a0c45474fa0b4352823c34dd2c163061
  Resolved https://github.com/terrier-org/pyterrier.git to commit 81f20bda483f6044bda89f717ae863e62bab42e5
  Preparing metadata (setup.py) ... done


In [ ]:
#Initialization de JVM
import pyterrier as pt
if not pt.started():
  pt.init()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8



### Lecture d'un index existant.

Ajoutez le code pour charger l'index créé sur **text** dans le TP 1.

In [ ]:
# 1. Ajoutez votre pd_index.zip du TP1 à Google Colab

# 2. Changer NOM REPERTOIRE par le nom de votre dossier. Après exécuté la commande suivante
!unzip pd_index.zip

Archive:  pd_index.zip
   creating: pd_index/
  inflating: pd_index/data.meta.idx  
  inflating: pd_index/data.meta-0.fsomapfile  
  inflating: pd_index/data.lexicon.fsomapid  
  inflating: pd_index/data.direct.bf  
  inflating: pd_index/data.lexicon.fsomaphash  
  inflating: pd_index/data.properties  
  inflating: pd_index/data.meta.zdata  
  inflating: pd_index/data.lexicon.fsomapfile  
  inflating: pd_index/data.inverted.bf  
  inflating: pd_index/data.document.fsarrayfile  


In [ ]:
indexref2 = pt.autoclass("org.terrier.querying.IndexRef").of(os.path.join("/content/pd_index", "data.properties"))

# 1) Liaison avec le texte
En utilisant l’indexe du champ **text** de Wikipédia (comme dans le TP1) et la requête *« Thomas and Mario are strikers playing in Munich »* vérifiez si vous obtenez les pages Wikipédia respectives (comme le feraient Spotlight, Babelfy ou TagMe) :
* Thomas Müller
* Mario Gómez ou Mario Götze
* FC Bayern Munich

Voici la requete sur l'index mentionné :

In [ ]:
pt.BatchRetrieve(indexref2, wmodel="BM25", metadata=["docno","title","url"]).search("Thomas and Mario are strikers playing in Munich").head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,30.771667,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,30.306808,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,29.796582,Thomas and Mario are strikers playing in Munich
3,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,3,28.570535,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,28.192863,Thomas and Mario are strikers playing in Munich
5,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,5,27.400834,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,25.804927,Thomas and Mario are strikers playing in Munich
7,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,7,25.073465,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,24.388908,Thomas and Mario are strikers playing in Munich
9,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,9,23.780821,Thomas and Mario are strikers playing in Munich


J'obtiens la page Thomas Muller à la 35eme place avec un score de 18.7%.
Pour les pages Mario Gómez ou Mario Götze et FC Bayern Munich elles n'apparaissent pas dans les 50 premiers résultats.

ICI VOTRE RÉPONSE POUR LA COMPARAISON DES RÉSULTATS

Babelfy : on retrouve les 3 pages demandées

TagMe :

# 2) Les poids

Utilisez les différentes méthodes de pondération prédéfinies sur Pyterrier (http://terrier.org/docs/v5.2/javadoc/org/terrier/matching/models/package-summary.html) pour voir s’il existe des méthodes que vous donnent de meilleurs résultats.

Choisir 5 méthodes (à vous de décider lesquels) et les comparer.

C’est-à-dire inspectez les 10 premiers résultats et vérifiez si les pages d’intérêt (Thomas Müller, Mario Gómez ou Mario Götze et FC Bayern Munich) font partie des résultats, puis les 20 premiers, et finalement les 50 premiers. Existent-ils de différences importantes ?

In [ ]:
# Définir la requête
query = "Thomas and Mario are strikers playing in Munich"

# Méthodes PL2
pt.BatchRetrieve(indexref2, wmodel = "PL2", metadata=["docno","title","url"]).search(query).head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,14.949048,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,14.685239,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,13.851380,Thomas and Mario are strikers playing in Munich
3,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,3,13.785998,Thomas and Mario are strikers playing in Munich
4,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,4,13.560615,Thomas and Mario are strikers playing in Munich
5,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,5,12.622423,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,12.323815,Thomas and Mario are strikers playing in Munich
7,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,7,12.071344,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,11.598371,Thomas and Mario are strikers playing in Munich
9,1,123459,575949,Tobias Kempe,https://simple.wikipedia.org/wiki/Tobias%20Kempe,9,11.100438,Thomas and Mario are strikers playing in Munich


In [ ]:
# Méthodes TF_IDF
pt.BatchRetrieve(indexref2, wmodel = "TF_IDF", metadata=["docno","title","url"]).search(query).head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,16.803635,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,16.550150,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,16.306862,Thomas and Mario are strikers playing in Munich
3,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,3,15.602268,Thomas and Mario are strikers playing in Munich
4,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,4,15.463170,Thomas and Mario are strikers playing in Munich
5,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,5,15.186105,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,14.196096,Thomas and Mario are strikers playing in Munich
7,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,7,13.780017,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,13.325713,Thomas and Mario are strikers playing in Munich
9,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,9,13.277452,Thomas and Mario are strikers playing in Munich


In [ ]:
# Méthodes PL2
pt.BatchRetrieve(indexref2, wmodel = "PL2", metadata=["docno","title","url"]).search(query).head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,14.949048,Thomas and Mario are strikers playing in Munich
1,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,1,14.685239,Thomas and Mario are strikers playing in Munich
2,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,2,13.851380,Thomas and Mario are strikers playing in Munich
3,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,3,13.785998,Thomas and Mario are strikers playing in Munich
4,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,4,13.560615,Thomas and Mario are strikers playing in Munich
5,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,5,12.622423,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,12.323815,Thomas and Mario are strikers playing in Munich
7,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,7,12.071344,Thomas and Mario are strikers playing in Munich
8,1,113762,509811,Audi Cup 2015,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,8,11.598371,Thomas and Mario are strikers playing in Munich
9,1,123459,575949,Tobias Kempe,https://simple.wikipedia.org/wiki/Tobias%20Kempe,9,11.100438,Thomas and Mario are strikers playing in Munich


In [ ]:
pt.BatchRetrieve(indexref2, wmodel = "InL2", metadata=["docno","title","url"]).search(query).head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,0,14.466667,Thomas and Mario are strikers playing in Munich
1,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,1,14.322309,Thomas and Mario are strikers playing in Munich
2,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,2,14.230277,Thomas and Mario are strikers playing in Munich
3,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,3,13.626481,Thomas and Mario are strikers playing in Munich
4,1,15782,60424,Donkey Kong,https://simple.wikipedia.org/wiki/Donkey%20Kong,4,13.513909,Thomas and Mario are strikers playing in Munich
5,1,166504,771849,Mario Balotelli,https://simple.wikipedia.org/wiki/Mario%20Balo...,5,13.354163,Thomas and Mario are strikers playing in Munich
6,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,6,12.553508,Thomas and Mario are strikers playing in Munich
7,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,7,12.169527,Thomas and Mario are strikers playing in Munich
8,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,8,12.146827,Thomas and Mario are strikers playing in Munich
9,1,123459,575949,Tobias Kempe,https://simple.wikipedia.org/wiki/Tobias%20Kempe,9,11.607190,Thomas and Mario are strikers playing in Munich


In [ ]:
pt.BatchRetrieve(indexref2, wmodel = "DFIC", metadata=["docno","title","url"]).search(query).head(50)

,qid,docid,docno,title,url,rank,score,query
0,1,13712,50727,Princess Daisy,https://simple.wikipedia.org/wiki/Princess%20D...,0,26.940586,Thomas and Mario are strikers playing in Munich
1,1,1066,4016,Mario,https://simple.wikipedia.org/wiki/Mario,1,26.698606,Thomas and Mario are strikers playing in Munich
2,1,74872,321537,Alexander Zickler,https://simple.wikipedia.org/wiki/Alexander%20...,2,25.732168,Thomas and Mario are strikers playing in Munich
3,1,13711,50725,Princess Peach,https://simple.wikipedia.org/wiki/Princess%20P...,3,25.183685,Thomas and Mario are strikers playing in Munich
4,1,7608,24698,Luigi,https://simple.wikipedia.org/wiki/Luigi,4,24.887361,Thomas and Mario are strikers playing in Munich
5,1,93934,409469,Charles Martinet,https://simple.wikipedia.org/wiki/Charles%20Ma...,5,24.595545,Thomas and Mario are strikers playing in Munich
6,1,71587,303588,Audi Cup 2013,https://simple.wikipedia.org/wiki/Audi%20Cup%2...,6,24.335201,Thomas and Mario are strikers playing in Munich
7,1,28329,114619,Mario Smash Football,https://simple.wikipedia.org/wiki/Mario%20Smas...,7,23.800121,Thomas and Mario are strikers playing in Munich
8,1,28330,114620,Mario Strikers Charged,https://simple.wikipedia.org/wiki/Mario%20Stri...,8,23.763274,Thomas and Mario are strikers playing in Munich
9,1,1074,4029,Yoshi,https://simple.wikipedia.org/wiki/Yoshi,9,23.688013,Thomas and Mario are strikers playing in Munich


Les resultats obtenuent ne sont pas très bon. La seule page d'intéret que je retrouve est Thomas Mller :

TF_IDF -> 33eme place

InL2 -> 22eme place

# 3) Utilisation de termes

Une stratégie consiste à utiliser individuellement chaque terme de la requête pour voir si vous arrivez à trouver les pages d’intérêt. Un code exemple est donné ci-dessous, cependant, il n'est pas suffisant pour faire la tâche.

Proposer une modification de cette fonction basique (à vous de vous inspirer/inventer des modifications qui améliorent les résultats)

Une fois fini votre code, réalisez la même vérification de résultats que dans le point précédent.


In [ ]:
def basic_term_search(query, method):
  termToSearch = [x for x in query.split() if x[0]==x[0].upper()]
  for term, page in zip(termToSearch,['Thomas Mller','Mario Gtze', 'FC Bayern Munich']):
    dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(term)[['docno','title','score','url']]
    display(dfres.head(5))
    print("Position de la page attendue dans les résultats ")
    display(dfres[dfres['title']==page].head(10))

query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")

,docno,title,score,url
0,401553,Thomas (surname),11.700841,https://simple.wikipedia.org/wiki/Thomas%20%28...
1,40378,Saint Thomas,11.597729,https://simple.wikipedia.org/wiki/Saint%20Thomas
2,27207,Thomas the Apostle,11.447421,https://simple.wikipedia.org/wiki/Thomas%20the...
3,773867,Thomas County,11.387148,https://simple.wikipedia.org/wiki/Thomas%20County
4,57053,Thomas Wedgwood IV,11.210921,https://simple.wikipedia.org/wiki/Thomas%20Wed...


Position de la page attendue dans les résultats 


,docno,title,score,url


,docno,title,score,url
0,50725,Princess Peach,16.478378,https://simple.wikipedia.org/wiki/Princess%20P...
1,77849,Mario (disambiguation),16.451796,https://simple.wikipedia.org/wiki/Mario%20%28d...
2,466220,Super Mario (disambiguation),16.419316,https://simple.wikipedia.org/wiki/Super%20Mari...
3,4016,Mario,16.395732,https://simple.wikipedia.org/wiki/Mario
4,409469,Charles Martinet,16.349001,https://simple.wikipedia.org/wiki/Charles%20Ma...


Position de la page attendue dans les résultats 


,docno,title,score,url
444,455348,Mario Gtze,6.193161,https://simple.wikipedia.org/wiki/Mario%20G%C3...


,docno,title,score,url
0,165793,Bundesliga,15.502280,https://simple.wikipedia.org/wiki/Bundesliga
1,869199,Olympiastadion (Munich),15.192906,https://simple.wikipedia.org/wiki/Olympiastadi...
2,2917,Munich,15.082408,https://simple.wikipedia.org/wiki/Munich
3,509811,Audi Cup 2015,14.993887,https://simple.wikipedia.org/wiki/Audi%20Cup%2...
4,403669,Technical University Munich,14.951113,https://simple.wikipedia.org/wiki/Technical%20...


Position de la page attendue dans les résultats 


,docno,title,score,url
177,6338,FC Bayern Munich,11.500917,https://simple.wikipedia.org/wiki/FC%20Bayern%...


In [ ]:
# ICI VOTRE CODE POUR LA COMPARAISON
def basic_term_search(query, method):

  terms = query.split()
  termToSearch = []

  for term in terms:
    if len(term) > 3:
        termToSearch.append(term)

  print(termToSearch)
# Effectuer la recherche en prenant les mots deux par deux
  for i in range(len(termToSearch) - 1):
      combined_query = termToSearch[i] + " " + termToSearch[i+1]
      print(combined_query)
      dfres = pt.BatchRetrieve(indexref2, wmodel=method, metadata=["docno","title","url"]).search(combined_query)[['title','score']]
      print(dfres.head(10))

  for page in ['Thomas Müller','Mario Götze', 'FC Bayern Munich']:
      print("Position de la page attendue dans les résultats :\n", dfres[dfres['title']==page],"\n\n\n")


query = "Thomas and Mario are strikers playing in Munich"
basic_term_search(query, "BM25")


['Thomas', 'Mario', 'strikers', 'playing', 'Munich']
Thomas Mario
                          title      score
0                Princess Peach  16.478378
1        Mario (disambiguation)  16.451796
2  Super Mario (disambiguation)  16.419316
3                         Mario  16.395732
4              Charles Martinet  16.349001
5                    Bowser Jr.  16.214048
6                         Wario  16.177717
7                         Luigi  16.132403
8                   Mario Bros.  15.990315
9                Princess Daisy  15.961688
Mario strikers
                    title      score
0  Mario Strikers Charged  30.771667
1    Mario Smash Football  30.306808
2             Donkey Kong  28.570535
3          Princess Daisy  27.133114
4                   Yoshi  24.164655
5         Mario Balotelli  23.841204
6          Princess Peach  23.706325
7                   Wario  21.020281
8                   Luigi  20.749609
9                   Mario  19.560429
strikers playing
                 title

....

ICI VOTRE RÉPONSE

....

# 4) Le titre
Maintenant utilisez le champ ***title*** avec la requête complète et avec chaque terme. Expliquez  pourquoi les résultats sont différents.

In [ ]:
# ICI VOTRE CODE POUR CHARGER L'AUTRE INDEX
!unzip pd_index5.zip
indexref5 = pt.autoclass("org.terrier.querying.IndexRef").of(os.path.join("/content/pd_index5", "data.properties"))


Archive:  pd_index5.zip
replace pd_index5/data.lexicon.fsomapid? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# Termes à rechercher
terms_to_search = ['Thomas Müller', 'Mario Götze', 'FC Bayern Munich']

# Recherche avec le modèle de pondération BM25
bm25 = pt.BatchRetrieve(indexref5, wmodel="BM25", metadata=["docno", "text", "url"])
results = bm25.search(" ".join(terms_to_search))

# Affichage des résultats et position des pages
for term in terms_to_search:
    print(f"Term: {term}")
    term_results = results[results['text'].str.contains(term)]
    if not term_results.empty:
        positions = term_results.index.values.tolist()
        print(f"Positions: {positions}")
    else:
        print("Ce terme n'a pas été trouvé dans les résultats.")
    print()

22:07:10.872 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
Term: Thomas Müller
Ce terme n'a pas été trouvé dans les résultats.

Term: Mario Götze
Ce terme n'a pas été trouvé dans les résultats.

Term: FC Bayern Munich
Positions: [0, 1, 2, 3, 4, 5, 6, 208]



....

ICI VOTRE RÉPONSE

....

# 5) L'outil
Développez un outil/méthode/système capable de recevoir comme entrée un paragraphe de texte et comme sortie une liste ordonné de pages Wikipédia pour chaque unité lexical (token ou mot) dans le paragraphe.



---



En plus à l’information du TP, vous pouvez utiliser les ressources/idées suivants pour améliorer les résultats du système à développer :
- La combinaison de plusieurs méthodes de pondération c’est possible en utilisant les indications sur https://pyterrier.readthedocs.io/en/latest/pipeline_examples.html .
- Comme mentionné dans le TP1, il existe plusieurs type de champs. On a utilisé celui de la classe TextField. Renseignez vous sur les autres (StringField par exemple) et ses différences.
- Les distances d’édition sont largement utilisés par les moteurs de recherche. Par exemple si vous tapez « paris sain germain » sur Google il va vous suggérer la requête « paris saint germain » l’équipe de football. Vous pouvez utiliser les distances d’édition pour trouver des entités avec une orthographe différent que celle sur Wikipédia. Une de distance très performantes est la distance Levenshtein implémentée dans la classe LuceneLevenshteinDistance du paquet lucene.search.spell.
- Les moteurs de recherche peuvent être utilisés pour calculer la similarité entre les mots ou entre les entités. Par exemple, sur Google si vous tapez « orange » et « carrot » il y a plus de résultats que si vous tapez « purple » et « carrot ». Vous pouvez utiliser cette information pour décider si deux pages Wikipédia font partie d’un même sujet.
- Utilisez le fichier [anchor_text_en.ttl](http://downloads.dbpedia.org/2015-10/core-i18n/en/anchor_text_en.ttl.bz2) qui contient les anchor texts  de la Wikipédia en anglais. Le format du fichier est décrit [ici](https://iai-group.github.io/DBpedia-Entity/index_details.html). Transformez-le dans ce format au autre similaire pour faire l'indexation:

|             Frequency                 	|       Wikipedia Page        	| Anchor Text 	|
|------------------------------	|---------------	|-----------	|
|162| \<Saturn_Award_for_Best_Supporting_Actor> |"Best Supporting Actor"|
|14 |\<André_Golke>| "Golke"|
|13 |\<Charles_Pinckney_National_Historic_Site> |"Charles Pinckney National Historic Site"|
|25 |\<Thomas_Manfredini>| "Manfredini"|
|13 |<Marek_Zieńczuk> |"Marek Zieńczuk"|
|14 |\<Sigurd_Ulveseth>| "Sigurd Ulveseth"|
|104| <Pole_Position_(video_game)>| "Pole Position"|
|10| <Bang_(band)>| "Bang"|
|44| <George_Susce_(catcher)>| "George Susce"|
|11| \<Kyle_Rankin>| "Kyle Rankin"|
|…|…|…|

Donc, la premier colonne correspond au nombre de fois que la chaîne de caractères de la troisième colonne a été utilisé pour faire allusion à la page Wikipédia de la deuxième colonne. C’est-à-dire que "Best Supporting Actor" a été utilisé 162 fois dans Wikipédia pour faire allusion à la page Wikipédia https://en.wikipedia.org/wiki/Saturn_Award_for_Best_Supporting_Actor . Le fichier anchor_text_en.tll  contient les informations de la Wikipédia en anglais et pas de la Wikipédia simple, donc certain titres n’ont pas de correspondance.

In [ ]:
# ICI VOTRE CODE

def systeme(paragraph, model, index):
    res = []
    mots = paragraph.split()
    mots_majuscules = [mot for mot in mots if mot[0].isupper()]

    for mot in mots_majuscules:
        bm25 = pt.BatchRetrieve(index, wmodel=model, metadata=["docno", "title", "url"])
        results = bm25.search(mot).head(10)
        res.append(results)

    print(res)

In [ ]:
text = "Thomas and Mario are strikers playing in Munich"
systeme(text,"BM25",indexref2)

22:11:39.810 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - OutOfMemoryError: Structure meta reading data file directly from disk
[  qid   docid   docno                        title  \
0   1   91931  401553             Thomas (surname)   
1   1   11169   40378                 Saint Thomas   
2   1    8167   27207           Thomas the Apostle   
3   1  167355  773867                Thomas County   
4   1   15129   57053           Thomas Wedgwood IV   
5   1  199625  889084                 Sarah Thomas   
6   1   94201  410176                 Helen Thomas   
7   1   27604  111782             Gospel of Thomas   
8   1   99243  432463      Thomas White (merchant)   
9   1  162167  760013  Mallappally Marthoma Church   

                                                 url  rank      score   query  
0  https://simple.wikipedia.org/wiki/Thomas%20%28...     0  11.700841  Thomas  
1   https://simple.wikipedia.org/wiki/Saint%20Thomas     1  11.597729  Thomas  
2  https://simple.w